In [1]:
import pandas as pd
from tqdm import tqdm
import os
import time

import sys
sys.path.append('../mlrose')
from mlrose_hiive.generators.four_peaks_generator import FourPeaksGenerator
from mlrose_hiive import RHCRunner
from mlrose_hiive.fitness.four_peaks import FourPeaks 

In [2]:
ALGORITHM = 'rhc'
T_PCT = 0.4
PROBLEM_SIZE_LIST = [10, 30, 50, 100, 200, 500]
MAX_ITERATIONS = 10000
NUM_RUNS = 3

RESTARTS = 50

In [3]:
problem = FourPeaks(t_pct=0.1)
problem.evaluate([1]*9 + [0]*11)

31

In [4]:
def get_opt_fitness(problem_size, t_pct):
    R = (1 - t_pct)*problem_size
    if R == int(R):
        R = int(R) - 1
    else:
        R = int(R)
    return problem_size + R

In [5]:
df_path = f'metrics_problem_size/{ALGORITHM}.csv'
os.makedirs(os.path.dirname(df_path), exist_ok=True)
if not os.path.exists(df_path):
    all_df = pd.DataFrame(columns=[
        # runner columns
        'Iteration', 'Time', 'Fitness', 'FEvals', 'Restarts', 'max_iters', 'current_restart',
        # problem config
        'problem_size', 't_pct', 'max_attempts',  
        # run config
        'run_number', 'group_timestamp', 'total_runtime',
    ])
    all_df.to_csv(df_path, index=False)
all_df = pd.read_csv(df_path)

In [ ]:
for problem_size in PROBLEM_SIZE_LIST:
    print(f"problem_size: {problem_size}")

    MAX_ATTEMPTS = problem_size * 5

    if (
        (all_df['problem_size'] == problem_size) &
        (all_df['t_pct'] == T_PCT) &
        (all_df['Restarts'] == RESTARTS) &
        (all_df['max_attempts'] == MAX_ATTEMPTS)
    ).any():
        print('Already run')
        continue

    group_timestamp = int(time.time())
    time.sleep(1)
    for run_i in tqdm(range(NUM_RUNS)):
        t_start = time.perf_counter()
        problem = FourPeaksGenerator().generate(seed=run_i, size=problem_size, t_pct=T_PCT)
        runner = RHCRunner(
            problem=problem,
            experiment_name='RHC',
            output_directory='.',
            seed=run_i,
            iteration_list=[MAX_ITERATIONS],
            max_attempts=MAX_ATTEMPTS,
            restart_list=[RESTARTS],
        )
        _, df_run_curves = runner.run()

        df_run_curves['problem_size'] = problem_size
        df_run_curves['t_pct'] = T_PCT
        df_run_curves['max_attempts'] = MAX_ATTEMPTS

        df_run_curves['Restarts'] = RESTARTS
        df_run_curves['max_iters'] = MAX_ITERATIONS

        df_run_curves['run_number'] = run_i
        df_run_curves['group_timestamp'] = group_timestamp

        print(f"Max Fitness: {df_run_curves['Fitness'].max()}")
        print(f"Max Iteration: {df_run_curves['Iteration'].max()}")

        t_end = time.perf_counter()
        df_run_curves['total_runtime'] = t_end - t_start
        df_run_curves['optimal_fitness'] = get_opt_fitness(problem_size, T_PCT)

        all_df = pd.concat([all_df, df_run_curves])
    all_df.reset_index(inplace=True, drop=True)
    all_df.to_csv(df_path, index=False)

In [6]:
all_df.columns

Index(['Iteration', 'Time', 'Fitness', 'FEvals', 'Restarts', 'max_iters',
       'current_restart', 'problem_size', 't_pct', 'max_attempts',
       'run_number', 'group_timestamp', 'total_runtime', 'optimal_fitness'],
      dtype='object')

In [7]:
agg_df = pd.DataFrame()
for group_timestamp in all_df['group_timestamp'].unique():
    df = all_df[all_df['group_timestamp'] == group_timestamp]
    max_fit = df['Fitness'].max()
    total_iters = df['Iteration'].max()
    max_fevals = df['FEvals'].max()
    # df.drop(columns=['Fitness', 'Iteration', 'FEvals', 'Time'], inplace=True)
    df.drop(columns=['Fitness', 'Iteration', 'FEvals', 'Time', 'current_restart'], inplace=True)
    assert(len(df) > NUM_RUNS)
    df.drop_duplicates(inplace=True)
    assert(len(df) == NUM_RUNS)
    df['max_fitness'] = max_fit
    df['total_iterations'] = total_iters
    df['max_fevals'] = max_fevals
    agg_df = pd.concat([agg_df, df])
agg_df.reset_index(inplace=True, drop=True)

In [8]:
df_path = f'metrics_problem_size/{ALGORITHM}_agg.csv'
agg_df.to_csv(df_path, index=False)